In [150]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import three_arb_check, one_iv_computation 

black_scholes = three_arb_check.black_scholes
get_spot_and_risk_free_rate = one_iv_computation.get_spot_and_risk_free_rate 
IV_surface_calls = pd.read_pickle('IV_surface_calls.pkl')
IV_surface_puts = pd.read_pickle('IV_surface_puts.pkl')

In [151]:
def calculate_greeks(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        delta = norm.cdf(d1)
    else:
        delta = norm.cdf(d1) - 1
    
    gamma = norm.pdf(d1)/(S * sigma * np.sqrt(T))
    tau = S * norm.pdf(d1) * np.sqrt(T) / 100
    if option_type == 'call':
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) 
                 - r * K * np.exp(-r * T) * norm.cdf(d2)) / 365
    else:
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) 
                 + r * K * np.exp(-r * T) * norm.cdf(-d2)) / 365
    if option_type == 'call':
        rho = K * T * np.exp(-r * T) * norm.cdf(d2) / 100
    else:
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2) / 100
    
    return {'delta': delta,'gamma': gamma,'tau': tau,'theta': theta,'rho': rho}


In [152]:
class Option:
    def __init__(self, strike, dte, option_type):
        self.strike = float(strike)
        self.dte = int(dte)
        self.option_type = str(option_type).lower()

    @property
    def T(self):
        return self.dte / 365.0

    @property
    def id(self):
        return f"{self.option_type}_{self.strike}_{self.dte}"

    def __repr__(self):
        return f"Option(type={self.option_type}, K={self.strike}, dte={self.dte})"

In [153]:
def get_mm_table(iv_grid_df, spot, T_in_years=True):
    # iv_grid_df: [moneyness, T, iv] 
    surface = (iv_grid_df.stack(dropna=True).rename("iv").reset_index())

    surface.columns = ["moneyness", "T", "iv"]

    surface["strike"] = surface["moneyness"] * spot

    if T_in_years:
        surface["dte"] = surface["T"] * 365
    else:
        surface["dte"] = surface["T"]

    surface["strike"] = surface["strike"].round(0).astype(int)
    surface["dte"] = surface["dte"].round(0).astype(int)

    mm_table = surface[["strike", "dte", "iv"]]

    return mm_table.dropna().drop_duplicates()

In [154]:
class MarketMaker:    
    def __init__(self, call_surface_df, put_surface_df, spot_price, risk_free_rate=0.05):
        #dfs: [strike, dte, iv]
        self.call_surface = call_surface_df
        self.put_surface = put_surface_df
        self.spot_price = spot_price
        self.risk_free_rate = risk_free_rate
        self.inventory = {}
        
        self.base_spread = 0.01
        self.inventory_skew_factor = 0.001
        self.delta_limit = 1000
        self.tau_limit = 1000
        self.delta_penalty_rate = 0.0001 
        self.tau_penalty_rate = 0.0001

        # I chose these numbers somewhat arbitrarily. 

        self.call_iv = (self.call_surface.set_index(["strike","dte"])["iv"])
        self.put_iv = (self.put_surface.set_index(["strike","dte"])["iv"])

        self.call_iv_dict = self.call_iv.to_dict()
        self.put_iv_dict = self.put_iv.to_dict()

        self.greeks = {}
        self.port = {"delta": 0.0, "gamma": 0.0, "tau": 0.0, "theta": 0.0, "rho": 0.0}
    
    def get_iv(self, option):
        key = (float(option.strike), int(option.dte))
        if option.option_type == "call": 
            return self.call_iv_dict.get(key)
        else:
            return self.put_iv_dict.get(key)
    
    def get_theo(self, option):
        iv = self.get_iv(option)
        return black_scholes(self.spot_price, option.strike, option.T, 
                            self.risk_free_rate, iv, option.option_type)
    
    def get_greeks(self, option):
        if option.id in self.greeks:
            return self.greeks[option.id]
        iv = self.get_iv(option)
        g = calculate_greeks(self.spot_price, option.strike, option.T, self.risk_free_rate, iv, option.option_type)
        self.greeks[option.id] = g
        return g 
    
    def port_greeks(self):
        return self.port.copy() 
    
    def limit_penalty(self,current_exposure,trade_exposure,limit,penalty_rate,activation_fraction=0.7):

        if abs(current_exposure) <= activation_fraction * limit:
            return 0.0

        exposure_ratio = abs(current_exposure) / limit
        return penalty_rate * abs(trade_exposure) * exposure_ratio
    
    def delta_penalty(self, option):
        greeks = self.get_greeks(option)
        portfolio_greeks = self.port_greeks()
        
        current_delta = portfolio_greeks['delta']
        option_delta = greeks['delta'] * 100  # per contract
        
        return self.limit_penalty(current_delta,option_delta,self.delta_limit,self.delta_penalty_rate)
    
    def tau_penalty(self, option):
        greeks = self.get_greeks(option)
        portfolio_greeks = self.port_greeks()
        
        current_tau = portfolio_greeks['tau']
        option_tau = greeks['tau'] * 100  # per contract
        
        return self.limit_penalty(current_tau,option_tau,self.tau_limit,self.tau_penalty_rate)
    
    def quote(self, option):
        # quote = theo \pm skew \pm Greek penalties \pm half spread 
        
        theo = self.get_theo(option)
        half_spread = self.base_spread * abs(theo)
        position = self.inventory.get(option.id, 0)
        skew = -self.inventory_skew_factor * theo * position

        delta_penalty = self.delta_penalty(option)
        tau_penalty = self.tau_penalty(option)

        if skew - delta_penalty - tau_penalty > theo - half_spread:
            bid = theo-half_spread
        else:
            bid = theo - half_spread + skew - delta_penalty - tau_penalty 

        ask = theo + half_spread + skew + tau_penalty + delta_penalty

        return bid, ask, theo
    
    def trade(self, option, side, quantity=1):
        sign = 1 if side == "buy" else -1 
        self.inventory[option.id] = self.inventory.get(option.id, 0)+sign*quantity 

        g = self.get_greeks(option)
        multiplier = sign*quantity*100 

        self.port["delta"] += multiplier * g["delta"]
        self.port["gamma"] += multiplier * g["gamma"]
        self.port["tau"] += multiplier * g["tau"]
        self.port["theta"] += multiplier * g["theta"]
        self.port["rho"] += multiplier * g["rho"]
        
    
    def get_inventory_summary(self):
        if not self.inventory:
            return None
        
        rows = []
        for option_id, position in self.inventory.items():
            if position == 0:
                continue
                
            parts = option_id.split('_')
            option = Option(float(parts[1]),int(parts[2]),parts[0])
            
            theo = self.get_theo(option)
            greeks = self.get_greeks(option)
            
            rows.append([option_id, position, option.strike, option.dte, theo, greeks])
        
        return rows 
    


In [155]:
def order_and_trade(strike, dte, type, position, quantity, mm):
    order = Option(strike, dte, type)
    b, a, t = mm.quote(order)
    print(f"Bid: ${b:.2f}, Ask: ${a:.2f}, Theo: ${t:.2f}")
    mm.trade(order, position, quantity)

def print_port_summary(greeks):
    print("\nSummary of portfolio greeks:")
    print(f"Delta: {greeks['delta']}")
    print(f"Gamma: {greeks['gamma']}")
    print(f"Theta: {greeks['theta']}")
    print(f"Tau: {greeks['tau']}")
    print(f"Rho: {greeks['rho']}")

def print_invt_summary(rows):
    print("\n Summary of options traded")
    for row in rows:
        print(f"option traded: {row[0]}, quantity: {row[1]}")

    
spot, rfr = get_spot_and_risk_free_rate()

call_surface_df = get_mm_table(IV_surface_calls, spot)
put_surface_df  = get_mm_table(IV_surface_puts,  spot)

mm = MarketMaker(call_surface_df, put_surface_df, spot, rfr)


In [156]:
# quickly check spread size
test = Option(695, 5, "call")

b, a, t = mm.quote(test)
print(f"ATM spread: {(a-b)/t*100:.1f}%")

ATM spread: 2.0%


In [157]:
order_and_trade(700, 5, "put", "sell", 5, mm)
order_and_trade(740, 10, "call", "sell", 10, mm)

print_invt_summary((mm.get_inventory_summary()))
print_port_summary(mm.port_greeks())

Bid: $10.09, Ask: $10.30, Theo: $10.20
Bid: $0.02, Ask: $0.02, Theo: $0.02

 Summary of options traded
option traded: put_700.0_5, quantity: -5
option traded: call_740.0_10, quantity: -10

Summary of portfolio greeks:
Delta: 305.34782055651186
Gamma: -11.396962870450656
Theta: 320.9627851405149
Tau: -166.41455139594657
Rho: 29.421657408852386
